In [1]:
import pandas as pd
from bokeh.io import output_notebook, show, export_png, export_svg
from bokeh.plotting import figure
from bokeh.models import BoxAnnotation
pd.options.display.max_colwidth = -1

/var/folders/pz/5lsvsk5s79q_8bbzj9jj5vv00000gn/T/ipykernel_80355/2510942243.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.options.display.max_colwidth = -1


In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
df = pd.read_csv("../data/final_twitter_data.csv", sep='\t', lineterminator='\n', error_bad_lines=False)

/var/folders/pz/5lsvsk5s79q_8bbzj9jj5vv00000gn/T/ipykernel_80355/2853051814.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv("../data/final_twitter_data.csv", sep='\t', lineterminator='\n', error_bad_lines=False)
/var/folders/pz/5lsvsk5s79q_8bbzj9jj5vv00000gn/T/ipykernel_80355/2853051814.py:1: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/final_twitter_data.csv", sep='\t', lineterminator='\n', error_bad_lines=False)


In [4]:
df.shape, df['author_id'].nunique()

((262578, 14), 16710)

In [5]:
df.created_at.min(), df.created_at.max()

('2010-02-12 16:19:32+00:00', '2022-08-08T04:58:25.000Z')

In [6]:
df.created_at = pd.to_datetime(df.created_at)
#df.created_at = df.created_at.dt.date
df['year'] = df.created_at.dt.year
df['month'] = df.created_at.dt.month
df['day'] = df.created_at.dt.day
df['month'] = df.month.apply(lambda x: int((x-1)//3)+1)
df['date'] = df.apply(lambda x: "{}/{}/{}".format(str(x.day),str(x.month),str(x.year)),axis=1)
df['date2'] = df.apply(lambda x: "{}/{}".format(str(x.month),str(x.year)),axis=1)

df = df.sort_values(['date']).reset_index(drop=True)

In [7]:
aggregated_data = df[df.CMI < 0.5].groupby(['date'])['id'].count().reset_index(drop=False)
aggregated_data.columns = ['date','Num_mono']

aggregated_data2 = df[df.CMI > 0.5].groupby(['date'])['id'].count().reset_index(drop=False)
aggregated_data2.columns = ['date','Num_CM']

aggregated_data3 = df.groupby(['date'])['id'].count().reset_index(drop=False)
aggregated_data3.columns = ['date','Total']

aggregated_data = pd.merge(aggregated_data, aggregated_data2, how='inner')
aggregated_data = pd.merge(aggregated_data, aggregated_data3, how='inner')

aggregated_data['day'] = aggregated_data.date.apply(lambda x: int(x.split("/")[0]))
aggregated_data['month'] = aggregated_data.date.apply(lambda x: int(x.split("/")[1]))
aggregated_data['year'] = aggregated_data.date.apply(lambda x: int(x.split("/")[2]))

aggregated_data = aggregated_data[aggregated_data.year >= 2016]
aggregated_data = aggregated_data.sort_values(['year','month','day']).reset_index(drop=True)

aggregated_data['Perc_mono'] = aggregated_data.Num_mono/aggregated_data.Total
aggregated_data['Perc_CM'] = aggregated_data.Num_CM/aggregated_data.Total
aggregated_data['CM_logit'] = aggregated_data.Num_CM/aggregated_data.Num_mono
aggregated_data['id'] = aggregated_data.index

In [8]:
import matplotlib.pyplot as plt
import math

In [9]:
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from dateutil.parser import parse
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.stattools import kpss

In [10]:
ts = df.groupby(['date2'])['CMI'].mean().reset_index(drop=False)
ts['month'] = ts.date2.apply(lambda x: int(x.split("/")[0]))
ts['year'] = ts.date2.apply(lambda x: int(x.split("/")[1]))
ts = ts.sort_values(['year','month']).reset_index(drop=True)
#ts.set_index(ts['date'], inplace = True)

In [11]:
ts2 = df.groupby(['date2'])['CMI'].std().reset_index(drop=False)
ts2['month'] = ts2.date2.apply(lambda x: int(x.split("/")[0]))
ts2['year'] = ts2.date2.apply(lambda x: int(x.split("/")[1]))
ts2 = ts2.sort_values(['year','month']).reset_index(drop=True)

In [12]:
import numpy as np

In [13]:
ts = ts.iloc[16:].reset_index(drop=True)
ts2 = ts2.iloc[16:].reset_index(drop=True)

In [14]:
ts

,date2,CMI,month,year
0,1/2014,0.556281,1,2014
1,2/2014,0.553769,2,2014
2,3/2014,0.524359,3,2014
3,4/2014,0.512680,4,2014
4,1/2015,0.479296,1,2015
5,2/2015,0.512120,2,2015
6,3/2015,0.497496,3,2015
7,4/2015,0.509839,4,2015
8,1/2016,0.487622,1,2016
9,2/2016,0.506850,2,2016


In [15]:
ts2

,date2,CMI,month,year
0,1/2014,0.217872,1,2014
1,2/2014,0.190510,2,2014
2,3/2014,0.181410,3,2014
3,4/2014,0.199198,4,2014
4,1/2015,0.195676,1,2015
5,2/2015,0.196554,2,2015
6,3/2015,0.201108,3,2015
7,4/2015,0.195575,4,2015
8,1/2016,0.189577,1,2016
9,2/2016,0.199247,2,2016


In [16]:
ts2['lower'] = ts.CMI - ts2.CMI
ts2['upper'] = ts.CMI + ts2.CMI

In [17]:
ts2

,date2,CMI,month,year,lower,upper
0,1/2014,0.217872,1,2014,0.338410,0.774153
1,2/2014,0.190510,2,2014,0.363259,0.744279
2,3/2014,0.181410,3,2014,0.342949,0.705769
3,4/2014,0.199198,4,2014,0.313482,0.711878
4,1/2015,0.195676,1,2015,0.283619,0.674972
5,2/2015,0.196554,2,2015,0.315567,0.708674
6,3/2015,0.201108,3,2015,0.296389,0.698604
7,4/2015,0.195575,4,2015,0.314264,0.705414
8,1/2016,0.189577,1,2016,0.298045,0.677200
9,2/2016,0.199247,2,2016,0.307603,0.706096


In [18]:
ts = ts.reset_index(drop=True)
y = np.array(ts.CMI.tolist())

In [19]:
n_breaks=2

In [20]:
import jenkspy
breaks = jenkspy.jenks_breaks(y, n_classes=n_breaks-1)

In [21]:
breaks_jkp = []
for v in breaks:
    idx = ts.index[ts.CMI == v]
    breaks_jkp.append(idx)

In [22]:
breaks_jkp

[Int64Index([4], dtype='int64'), Int64Index([25], dtype='int64')]

In [23]:
ts.iloc[:4]

,date2,CMI,month,year
0,1/2014,0.556281,1,2014
1,2/2014,0.553769,2,2014
2,3/2014,0.524359,3,2014
3,4/2014,0.512680,4,2014


In [24]:
ts.iloc[25:]

,date2,CMI,month,year
25,2/2020,0.567325,2,2020
26,3/2020,0.556010,3,2020
27,4/2020,0.552498,4,2020
28,1/2021,0.553584,1,2021
29,2/2021,0.565688,2,2021
30,3/2021,0.553620,3,2021
31,4/2021,0.553945,4,2021
32,1/2022,0.553522,1,2022
33,2/2022,0.564801,2,2022
34,3/2022,0.560430,3,2022


In [25]:
df.month.value_counts()

2    81638
3    74343
1    60033
4    46564
Name: month, dtype: int64

In [26]:
df.head()

,created_at,author_id,lang,text,id,public_metrics,withheld,script_lang,translated_text,POS,LID,CMI,Langugage_hueristic,follower_count,year,month,day,date,date2
0,2014-03-01 10:28:08+00:00,2.371767e+09,hi,अपन देशक नेता जी : कविता: चोर उचक्का नेता बनी केघोटाला पर जे घोटाला करतघैयला म जे भुर रहत ततहन पैइन कोना क भरत... http://t.co/V4RC5FUSbx,4.397087e+17,"{'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}",NaN,hindi,apan deshak netaa jee : kavita: chor uchakka netaa bunny keghotala parr jey ghotala karatghaiyala maam jey bhur rahat tatahan paine cona kaa bharat... http://t.co/v4rc5fusbx,"{""apan"": ""ADV"", ""deshak"": ""NOUN"", ""netaa"": ""VERB"", ""jee"": ""VERB"", "":"": ""X"", ""kavita"": ""NOUN"", ""chor"": ""NOUN"", ""uchakka"": ""VERB"", ""bunny"": ""NOUN"", ""keghotala"": ""VERB"", ""parr"": ""PART"", ""jey"": ""PRON"", ""ghotala"": ""VERB"", ""karatghaiyala"": ""VERB"", ""maam"": ""ADP"", ""bhur"": ""VERB"", ""rahat"": ""VERB"", ""tatahan"": ""NOUN"", ""paine"": ""VERB"", ""cona"": ""VERB"", ""kaa"": ""VERB"", ""bharat"": ""PROPN"", ""."": ""X"", ""http"": ""X"", ""/"": ""X"", ""t"": ""X"", ""co"": ""X"", ""v4rc5fusbx"": ""X""}","{""apan"": ""hin"", ""deshak"": ""hin"", ""netaa"": ""hin"", ""jee"": ""hin"", "":"": ""other"", ""kavita"": ""hin"", ""chor"": ""hin"", ""uchakka"": ""hin"", ""bunny"": ""hin"", ""keghotala"": ""hin"", ""parr"": ""hin"", ""jey"": ""hin"", ""ghotala"": ""hin"", ""karatghaiyala"": ""hin"", ""maam"": ""hin"", ""bhur"": ""hin"", ""rahat"": ""hin"", ""tatahan"": ""hin"", ""paine"": ""hin"", ""cona"": ""hin"", ""kaa"": ""hin"", ""bharat"": ""ne"", ""."": ""other"", ""http"": ""other"", ""/"": ""other"", ""t"": ""en"", ""co"": ""en"", ""v4rc5fusbx"": ""other""}",0.714286,Code-mixed,1.0,2014,1,1,1/1/2014,1/2014
1,2014-03-01 20:37:07+00:00,1.311165e+08,hi,Shri. Pranab Mukherjee: Do not change the name of Shankaracharya Hill in Kashmir http://t.co/NV6fL7SdEH via @Change4India,4.398619e+17,"{'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}",NaN,NaN,Shri. Pranab Mukherjee: Do not change the name of Shankaracharya Hill in Kashmir http://t.co/NV6fL7SdEH via @Change4India,"{""Shri"": ""PROPN"", ""."": ""X"", ""Pranab"": ""PROPN"", ""Mukherjee"": ""PROPN"", "":"": ""X"", ""Do"": ""VERB"", ""not"": ""PART_NEG"", ""change"": ""VERB"", ""the"": ""DET"", ""name"": ""NOUN"", ""of"": ""ADP"", ""Shankaracharya"": ""PROPN"", ""Hill"": ""PROPN"", ""in"": ""ADP"", ""Kashmir"": ""PROPN"", ""http"": ""X"", ""/"": ""X"", ""t"": ""X"", ""co"": ""X"", ""NV6fL7SdEH"": ""X"", ""via"": ""ADP"", ""@"": ""X"", ""Change4India"": ""X""}","{""Shri"": ""ne"", ""."": ""other"", ""Pranab"": ""ne"", ""Mukherjee"": ""ne"", "":"": ""other"", ""Do"": ""en"", ""not"": ""en"", ""change"": ""en"", ""the"": ""en"", ""name"": ""en"", ""of"": ""en"", ""Shankaracharya"": ""ne"", ""Hill"": ""ne"", ""in"": ""en"", ""Kashmir"": ""ne"", ""http"": ""other"", ""/"": ""other"", ""t"": ""en"", ""co"": ""en"", ""NV6fL7SdEH"": ""en"", ""via"": ""en"", ""@"": ""other"", ""Change4India"": ""ne""}",0.478261,English,NaN,2014,1,1,1/1/2014,1/2014
2,2014-01-01 17:37:06+00:00,8.914634e+07,hi,ae dil kis baat par khush hae tu? Din to badal jaate hae...dil nahi ...a very happy 2014..cheers!,4.184357e+17,"{'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}",NaN,NaN,ae dil kis baat par khush hae tu? Din to badal jaate hae...dil nahi ...a very happy 2014..cheers!,"{""ae"": ""DET"", ""dil"": ""NOUN"", ""kis"": ""ADJ"", ""baat"": ""NOUN"", ""par"": ""ADP"", ""khush"": ""VERB"", ""hae"": ""VERB"", ""tu"": ""PRON"", ""?"": ""X"", ""Din"": ""NOUN"", ""to"": ""PART"", ""badal"": ""VERB"", ""jaate"": ""VERB"", ""."": ""X"", ""nahi"": ""PART_NEG"", ""a"": ""DET"", ""very"": ""ADV"", ""happy"": ""ADJ"", ""2014"": ""NUM"", ""cheers"": ""VERB"", ""!"": ""X""}","{""ae"": ""hin"", ""dil"": ""hin"", ""kis"": ""hin"", ""baat"": ""hin"", ""par"": ""hin"", ""khush"": ""hin"", ""hae"": ""hin"", ""tu"": ""hin"", ""?"": 

In [27]:
df[df.year == 2014].shape, df[df.year == 2014].author_id.nunique(), df[df.year == 2014].CMI.median()

((1502, 19), 515, 0.5238095238095238)

In [29]:
dff = pd.concat([df[(df.year > 2014) & (df.year <= 2019)], df[(df.year == 2020) & (df.month == 1)]],0)
dff.shape, dff.author_id.nunique(), dff.CMI.median()

/var/folders/pz/5lsvsk5s79q_8bbzj9jj5vv00000gn/T/ipykernel_80355/2680671534.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  dff = pd.concat([df[(df.year > 2014) & (df.year <= 2019)], df[(df.year == 2020) & (df.month == 1)]],0)


((43877, 19), 7282, 0.5294117647058824)

In [30]:
dff = pd.concat([df[df.year > 2020], df[(df.year == 2020) & (df.month > 1)]],0)
dff.shape, dff.author_id.nunique(), dff.CMI.median()

/var/folders/pz/5lsvsk5s79q_8bbzj9jj5vv00000gn/T/ipykernel_80355/3959322942.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  dff = pd.concat([df[df.year > 2020], df[(df.year == 2020) & (df.month > 1)]],0)


((217120, 19), 11112, 0.5660377358490566)

In [21]:
from bokeh.io import export_png
from bokeh.models import Band, ColumnDataSource, Range1d, Whisker, LinearAxis, SingleIntervalTicker, Label

In [22]:
l = []
for year in [2014,2015,2016,2017,2018,2019,2020,2021,2022]:
    for month in [1,2,3,4]:
        l.append("{}/{}".format(month,year))

l = l[:-1]

p = figure(height=600, width=1200, x_range=l)


cats = ['2/2014','2/2015','2/2016','2/2017','2/2018','2/2019','2/2020','2/2021','2/2022'] #ts.date2.to_list()

#groups = df[df.date2.str.contains("|".join(cats))].groupby('date2')
groups = df[df.date2.str.contains("|".join(cats))].groupby('year')

q1 = groups['CMI'].quantile(q=0.25).reset_index(drop=False)
#q1['month'] = q1.date2.apply(lambda x: int(x.split("/")[0]))
#q1['year'] = q1.date2.apply(lambda x: int(x.split("/")[1]))
#q1 = q1.sort_values(['year','month']).reset_index(drop=True)

q2 = groups['CMI'].quantile(q=0.5).reset_index(drop=False)

#q2['month'] = q2.date2.apply(lambda x: int(x.split("/")[0]))
#q2['year'] = q2.date2.apply(lambda x: int(x.split("/")[1]))
#q2 = q2.sort_values(['year','month']).reset_index(drop=True)

q3 = groups['CMI'].quantile(q=0.75).reset_index(drop=False)

#q3['month'] = q3.date2.apply(lambda x: int(x.split("/")[0]))
#q3['year'] = q3.date2.apply(lambda x: int(x.split("/")[1]))
#q3 = q3.sort_values(['year','month']).reset_index(drop=True)

iqr = q3.copy()
iqr['CMI'] = q3['CMI'] - q1['CMI']
upper = q3.copy()
upper['CMI'] = q3['CMI'] + 1*iqr['CMI']
upper.CMI = upper.CMI.clip(0,1)
lower = q3.copy()
lower['CMI'] = q1['CMI'] - 1*iqr['CMI']
lower.CMI = lower.CMI.clip(0,1)

# if no outliers, shrink lengths of stems to be no longer than the minimums or maximums
qmin = groups.quantile(q=0.00)
qmax = groups.quantile(q=1.00)
upper.CMI = [min([x,y]) for (x,y) in zip(list(qmax.loc[:,'CMI']),upper.CMI)]
lower.CMI = [max([x,y]) for (x,y) in zip(list(qmin.loc[:,'CMI']),lower.CMI)]

# stems
p.segment(cats, upper.CMI, cats, q3.CMI, line_color="black")
p.segment(cats, lower.CMI, cats, q1.CMI, line_color="black")

# boxes
p.vbar(cats, 1.5, q2.CMI, q3.CMI, fill_color="#E08E79", line_color="black")
p.vbar(cats, 1.5, q1.CMI, q2.CMI, fill_color="#3B8686", line_color="black")

# whiskers (almost-0 height rects simpler than segments)
p.rect(cats, lower.CMI, 0.2, 0.01, line_color="black")
p.rect(cats, upper.CMI, 0.2, 0.01, line_color="black")

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = 'Average CMI'

p.xaxis.axis_label_text_font_style = "normal"
p.yaxis.axis_label_text_font_style = "normal"

p.xaxis.axis_label_text_font_size = "20pt"
p.yaxis.axis_label_text_font_size = "20pt"

p.xaxis.major_label_text_font_size = "12pt"
p.yaxis.major_label_text_font_size = "12pt"

#p.xaxis.axis_tick_text_font_size = "20pt"
#p.yaxis.axis_tick_text_font_size = "20pt"

ticker = SingleIntervalTicker(interval=2, num_minor_ticks=10)
xaxis = LinearAxis(ticker=ticker)

p.output_backend = "svg"

label_dict= {}
for i in l:
    if i.split('/')[0] in ['1','3']:
        label_dict[i] = i
    else:
        label_dict[i] = ""
        
#p.xaxis[0].formatter = FixedTickFormatter(labels=label_dict)
#p.xaxis.major_label_overrides = label_dict 

#p.add_layout(xaxis, 'below')

p.line(ts.date2, ts.CMI,line_width=4)

source = ColumnDataSource(ts2.reset_index())

band = Band(base='date2', lower='lower', upper='upper', source=source, level='underlay', \
            fill_alpha=1.0, line_width=2, line_color='black')
p.add_layout(band)

p.xaxis.major_label_orientation = math.pi/4


lower = BoxAnnotation(right=breaks_jkp[0].values[0], fill_alpha=0.1, fill_color='firebrick')
p.add_layout(lower)

lower = BoxAnnotation(left=breaks_jkp[0].values[0], right=breaks_jkp[1].values[0], fill_alpha=0.1, fill_color='navy')
p.add_layout(lower)

lower = BoxAnnotation(left=breaks_jkp[1].values[0], fill_alpha=0.1, fill_color='olive')
p.add_layout(lower)

p.y_range = Range1d(0, 1)

label1 = Label(x=0.6, y=0.03, text='Phrase 1',text_font_size='15pt')
label2 = Label(x=15, y=0.03, text='Phrase 2',text_font_size='15pt')
label3 = Label(x=28.5, y=0.03, text='Phrase 3',text_font_size='15pt')

label4 = Label(x=15, y=0.22, text='Growth \n Stage',text_font_size='15pt',angle=20/90)
label5 = Label(x=30.5, y=0.22, text='Stable \n Stage',text_font_size='15pt',angle=0)

p.add_layout(label1)
p.add_layout(label2)
p.add_layout(label3)
p.add_layout(label4)
p.add_layout(label5)

#export_svg(p, filename="/Users/victor/Downloads/plot1.svg")

show(p)

/var/folders/pz/5lsvsk5s79q_8bbzj9jj5vv00000gn/T/ipykernel_14885/1605090259.py:43: FutureWarning: Dropping invalid columns in DataFrameGroupBy.quantile is deprecated. In a future version, a TypeError will be raised. Before calling .quantile, select only columns which should be valid for the function.
  qmin = groups.quantile(q=0.00)
/var/folders/pz/5lsvsk5s79q_8bbzj9jj5vv00000gn/T/ipykernel_14885/1605090259.py:44: FutureWarning: Dropping invalid columns in DataFrameGroupBy.quantile is deprecated. In a future version, a TypeError will be raised. Before calling .quantile, select only columns which should be valid for the function.
  qmax = groups.quantile(q=1.00)


### Statistical Tests

Run statistical tests to validate the change in CMI over years

In [23]:
from statsmodels.tsa.seasonal import seasonal_decompose
from dateutil.parser import parse
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.stattools import kpss

In [24]:
from sklearn.linear_model import LinearRegression

ll = LinearRegression(fit_intercept=True)
ll.fit(np.arange(20).reshape(-1, 1), ts.CMI.iloc[4:24])

LinearRegression()

In [25]:
ll.coef_, ll.intercept_

(array([0.00319146]), 0.4902630577436619)

In [27]:
ts['id'] = ts.index

In [28]:
X = np.arange(ts.shape[0]).reshape(-1, 1)
X = np.concatenate([X,np.asarray([1]*X.shape[0]).reshape(-1, 1)],-1)
mod = sm.OLS(ts.CMI, X)
res = mod.fit()
print(res.summary())
print (grangercausalitytests(ts[['CMI', 'id']], maxlag=2))

                            OLS Regression Results                            
Dep. Variable:                    CMI   R-squared:                       0.489
Model:                            OLS   Adj. R-squared:                  0.473
Method:                 Least Squares   F-statistic:                     31.56
Date:                Thu, 16 Mar 2023   Prob (F-statistic):           2.98e-06
Time:                        21:29:40   Log-Likelihood:                 92.336
No. Observations:                  35   AIC:                            -180.7
Df Residuals:                      33   BIC:                            -177.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0017      0.000      5.618      0.0

In [49]:
?grangercausalitytests

In [29]:
X = np.arange(20).reshape(-1, 1)
X = np.concatenate([X,np.asarray([1]*X.shape[0]).reshape(-1, 1)],-1)
mod = sm.OLS(ts.CMI.iloc[4:24], X)
res = mod.fit()
print(res.summary())
print (grangercausalitytests(ts[['CMI', 'id']].iloc[4:24], maxlag=2))

                            OLS Regression Results                            
Dep. Variable:                    CMI   R-squared:                       0.767
Model:                            OLS   Adj. R-squared:                  0.755
Method:                 Least Squares   F-statistic:                     59.41
Date:                Thu, 16 Mar 2023   Prob (F-statistic):           4.15e-07
Time:                        21:30:02   Log-Likelihood:                 63.467
No. Observations:                  20   AIC:                            -122.9
Df Residuals:                      18   BIC:                            -120.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0032      0.000      7.708      0.0

In [33]:
X = np.arange(4).reshape(-1, 1)
X = np.concatenate([X,np.asarray([1]*X.shape[0]).reshape(-1, 1)],-1)
mod = sm.OLS(ts.CMI.iloc[:4], X)
res = mod.fit()
print(res.summary())
print (grangercausalitytests(ts[['CMI', 'id']].iloc[:4], maxlag=1))

                            OLS Regression Results                            
Dep. Variable:                    CMI   R-squared:                       0.914
Model:                            OLS   Adj. R-squared:                  0.871
Method:                 Least Squares   F-statistic:                     21.29
Date:                Thu, 16 Mar 2023   Prob (F-statistic):             0.0439
Time:                        21:30:32   Log-Likelihood:                 15.143
No. Observations:                   4   AIC:                            -26.29
Df Residuals:                       2   BIC:                            -27.51
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.0160      0.003     -4.614      0.0

/Users/victor/miniforge3/lib/python3.9/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 4 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


ValueError: Insufficient observations. Maximum allowable lag is 0

In [34]:
X = np.arange(10).reshape(-1, 1)
X = np.concatenate([X,np.asarray([1]*X.shape[0]).reshape(-1, 1)],-1)
mod = sm.OLS(ts.CMI.iloc[25:], X)
res = mod.fit()
print(res.summary())
print (grangercausalitytests(ts[['CMI', 'id']].iloc[25:], maxlag=2))

                            OLS Regression Results                            
Dep. Variable:                    CMI   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.125
Method:                 Least Squares   F-statistic:                  0.003210
Date:                Thu, 16 Mar 2023   Prob (F-statistic):              0.956
Time:                        21:30:42   Log-Likelihood:                 37.762
No. Observations:                  10   AIC:                            -71.52
Df Residuals:                       8   BIC:                            -70.92
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1         -3.866e-05      0.001     -0.057      0.9

/Users/victor/miniforge3/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1477: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [36]:
ts['id'] = ts.index
for i in range(len(breaks_jkp)+1):
    if i == 0:
        print ("Between first timestep and {}".format(breaks_jkp[i].values[0]))

        ts2 = ts.iloc[0:breaks_jkp[i].values[0]].reset_index(drop=True)
        ts2['id'] = ts2.index
        
    elif i == len(breaks_jkp):
        print ("Between {} and last timestep".format(breaks_jkp[i-1].values[0]))

        ts2 = ts.iloc[breaks_jkp[i-1].values[0]:].reset_index(drop=True)
        ts2['id'] = ts2.index
        
    else:
        print ("Between {} and {} timestep".format(breaks_jkp[i-1].values[0], breaks_jkp[i].values[0]))

        ts2 = ts.iloc[breaks_jkp[i-1].values[0]:breaks_jkp[i].values[0]].reset_index(drop=True)
        ts2['id'] = ts2.index
    
    #additive_decomposition = seasonal_decompose(ts2.CMI, model='additive', period=4)
    #p = figure(height=400, width=1000, x_range=ts.date.unique())
    #p.line(ts2.date, additive_decomposition.trend, legend_label='CMI', line_dash=[4,4],line_width=2)
    #p.xaxis.major_label_orientation = math.pi/4
    #show(p)
    
    X = ts2['id'].values.reshape(-1,1)
    X = np.concatenate([X,np.asarray([1]*X.shape[0]).reshape(-1, 1)],-1)
    mod = sm.OLS(ts2.CMI, X)
    res = mod.fit()
    print(res.summary())
        
    #print (grangercausalitytests(ts2[['CMI', 'id']], maxlag=2))
    
    try:
        #print (kpss(ts2.CMI, regression='ct'))
        print (grangercausalitytests(ts2[['CMI', 'id']], maxlag=2))
    except:
        pass

Between first timestep and 4
                            OLS Regression Results                            
Dep. Variable:                    CMI   R-squared:                       0.914
Model:                            OLS   Adj. R-squared:                  0.871
Method:                 Least Squares   F-statistic:                     21.29
Date:                Thu, 16 Mar 2023   Prob (F-statistic):             0.0439
Time:                        21:33:11   Log-Likelihood:                 15.143
No. Observations:                   4   AIC:                            -26.29
Df Residuals:                       2   BIC:                            -27.51
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.0160  

/Users/victor/miniforge3/lib/python3.9/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 4 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
/Users/victor/miniforge3/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1477: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [37]:
import ast

In [38]:
df['Num_Eng'] = df.LID.apply(lambda x: list(ast.literal_eval(x).values()).count("en"))
df['Num_Hin'] = df.LID.apply(lambda x: list(ast.literal_eval(x).values()).count("hin"))

In [39]:
aggregated_data = df[(df.CMI < 0.5) & (df.Num_Eng > df.Num_Hin)].groupby(['date2'])['id'].count().reset_index(drop=False)
aggregated_data.columns = ['date2','Num_mono_Eng']

aggregated_data22 = df[(df.CMI < 0.5) & (df.Num_Eng < df.Num_Hin)].groupby(['date2'])['id'].count().reset_index(drop=False)
aggregated_data22.columns = ['date2','Num_mono_Hin']

aggregated_data2 = df[df.CMI >= 0.5].groupby(['date2'])['id'].count().reset_index(drop=False)
aggregated_data2.columns = ['date2','Num_CM']

aggregated_data3 = df.groupby(['date2'])['id'].count().reset_index(drop=False)
aggregated_data3.columns = ['date2','Total']

aggregated_data = pd.merge(aggregated_data, aggregated_data2, how='inner')
aggregated_data = pd.merge(aggregated_data, aggregated_data3, how='inner')
aggregated_data = pd.merge(aggregated_data, aggregated_data22, how='inner')

aggregated_data['month'] = aggregated_data.date2.apply(lambda x: int(x.split("/")[0]))
aggregated_data['year'] = aggregated_data.date2.apply(lambda x: int(x.split("/")[1]))
aggregated_data = aggregated_data.sort_values(['year','month']).reset_index(drop=True)

aggregated_data['Perc_mono_Hin'] = aggregated_data.Num_mono_Hin/aggregated_data.Total
aggregated_data['Perc_mono_Eng'] = aggregated_data.Num_mono_Eng/aggregated_data.Total
aggregated_data['Perc_CM'] = aggregated_data.Num_CM/aggregated_data.Total

aggregated_data = aggregated_data[aggregated_data.year >= 2014].reset_index(drop=True)

In [ ]:
p = figure(height=600, width=1200, x_range=aggregated_data.date2.unique())

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = 'Proportion'
#

p.y_range = Range1d(start=-0, end=1)
p.extra_y_ranges = {"Volume": Range1d(start=-0, end=50000)}

p.add_layout(LinearAxis(y_range_name="Volume", axis_label='Number of Tweets'), 'right')

p.vbar(x=aggregated_data.date2, top=aggregated_data.Total, bottom=0, width=0.5, y_range_name='Volume', color='orange')

p.line(aggregated_data.date2, aggregated_data.Perc_mono_Hin, legend_label='Monolingual Hindi', line_color="violet", line_dash=[4,4],line_width=4)
p.line(aggregated_data.date2, aggregated_data.Perc_CM, legend_label='Code-mixed',line_width=4)
p.line(aggregated_data.date2, aggregated_data.Perc_mono_Eng, legend_label='Monolingual English',line_color="black", line_width=4, line_dash=[2,2])

p.xaxis.major_label_orientation = math.pi/4

lower = BoxAnnotation(right=breaks_jkp[0].values[0], fill_alpha=0.1, fill_color='firebrick')
p.add_layout(lower)

lower = BoxAnnotation(left=breaks_jkp[0].values[0], right=breaks_jkp[1].values[0], fill_alpha=0.1, fill_color='navy')
p.add_layout(lower)

lower = BoxAnnotation(left=breaks_jkp[1].values[0], fill_alpha=0.1, fill_color='olive')
p.add_layout(lower)


#p.add_layout(p.legend[0], 'center')

p.xaxis.axis_label_text_font_style = "normal"
p.yaxis.axis_label_text_font_style = "normal"

p.legend.location = "top_left"
p.legend.click_policy="hide"

p.xaxis.axis_label_text_font_style = "normal"
p.yaxis.axis_label_text_font_style = "normal"

p.xaxis.axis_label_text_font_size = "20pt"
p.yaxis.axis_label_text_font_size = "20pt"

p.xaxis.major_label_text_font_size = "12pt"
p.yaxis.major_label_text_font_size = "12pt"

p.legend.title_text_font_size = '12pt'
 
# Increasing the labels of the box
# to 30pt
p.legend.label_text_font_size = "12pt"

p.output_backend = "svg"

label1 = Label(x=15, y=0.46, text='Increasing trend',text_font_size='15pt',angle=10/90)
label2 = Label(x=15, y=0.23, text='Decreasing trend',text_font_size='15pt',angle=-10/90)

p.add_layout(label1)
p.add_layout(label2)

export_svg(p, filename="/Users/victor/Downloads/figure2.svg")

show(p)


### Statistical Tests

In [41]:
aggregated_data['id'] = aggregated_data.index
for i in range(len(breaks_jkp)+1):
    if i == 0:
        print ("Between first timestep and {}".format(breaks_jkp[i].values[0]))

        aggregated_data2 = aggregated_data.iloc[0:breaks_jkp[i].values[0]].reset_index(drop=True)
        aggregated_data2['id'] = aggregated_data2.index
        
    elif i == len(breaks_jkp):
        print ("Between {} and last timestep".format(breaks_jkp[i-1].values[0]))

        aggregated_data2 = aggregated_data.iloc[breaks_jkp[i-1].values[0]:].reset_index(drop=True)
        aggregated_data2['id'] = aggregated_data2.index
        
    else:
        print ("Between {} and {} timestep".format(breaks_jkp[i-1].values[0], breaks_jkp[i].values[0]))

        aggregated_data2 = aggregated_data.iloc[breaks_jkp[i-1].values[0]:breaks_jkp[i].values[0]].reset_index(drop=True)
        aggregated_data2['id'] = aggregated_data2.index
    
    #additive_decomposition = seasonal_decompose(aggregated_data2.CMI, model='additive', period=4)
    #p = figure(height=400, width=1000, x_range=aggregated_data.date.unique())
    #p.line(aggregated_data2.date, additive_decomposition.trend, legend_label='CMI', line_dash=[4,4],line_width=2)
    #p.xaxis.major_label_orientation = math.pi/4
    #show(p)
    
    X = aggregated_data2['id'].values.reshape(-1,1)
    X = np.concatenate([X,np.asarray([1]*X.shape[0]).reshape(-1, 1)],-1)
    
    print("Mono Hindi")
    mod = sm.OLS(aggregated_data2.Perc_mono_Hin, X)
    res = mod.fit()
    print(res.summary())
    
    print("Mono English")
    mod = sm.OLS(aggregated_data2.Perc_mono_Eng, X)
    res = mod.fit()
    print(res.summary())
    
    print("CM")
    mod = sm.OLS(aggregated_data2.Perc_CM, X)
    res = mod.fit()
    print(res.summary())
    
    try:
        print ("Mono Hindi")
        #print (kpss(aggregated_data2.Perc_mono_Hin, regression='ct'))
        print (grangercausalitytests(aggregated_data2[['Perc_mono_Hin', 'id']], maxlag=2))
        
        print ("Mono English")
        #print (kpss(aggregated_data2.Perc_mono_Eng, regression='ct'))
        print (grangercausalitytests(aggregated_data2[['Perc_mono_Eng', 'id']], maxlag=2))
        
        print ("CM")
        #print (kpss(aggregated_data2.Perc_CM, regression='ct'))
        print (grangercausalitytests(aggregated_data2[['Perc_CM', 'id']], maxlag=2))
        
    except:
        pass

Between first timestep and 4
Mono Hindi
                            OLS Regression Results                            
Dep. Variable:          Perc_mono_Hin   R-squared:                       0.848
Model:                            OLS   Adj. R-squared:                  0.772
Method:                 Least Squares   F-statistic:                     11.16
Date:                Thu, 16 Mar 2023   Prob (F-statistic):             0.0791
Time:                        21:34:14   Log-Likelihood:                 11.376
No. Observations:                   4   AIC:                            -18.75
Df Residuals:                       2   BIC:                            -19.98
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          

/Users/victor/miniforge3/lib/python3.9/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 4 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
/Users/victor/miniforge3/lib/python3.9/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 4 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
/Users/victor/miniforge3/lib/python3.9/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 4 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
/Users/victor/miniforge3/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1477: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/Users/victor/

In [42]:
X = aggregated_data['id'].values.reshape(-1,1)
X = np.concatenate([X,np.asarray([1]*X.shape[0]).reshape(-1, 1)],-1)
    
print("Mono Hindi")
mod = sm.OLS(aggregated_data.Perc_mono_Hin, X)
res = mod.fit()
print(res.summary())
print (grangercausalitytests(aggregated_data[['Perc_mono_Hin', 'id']], maxlag=2))

print("Mono English")
mod = sm.OLS(aggregated_data.Perc_mono_Eng, X)
res = mod.fit()
print(res.summary())
print (grangercausalitytests(aggregated_data[['Perc_mono_Eng', 'id']], maxlag=2))

print("CM")
mod = sm.OLS(aggregated_data.Perc_CM, X)
res = mod.fit()
print(res.summary())
print (grangercausalitytests(aggregated_data[['Perc_CM', 'id']], maxlag=2))

Mono Hindi
                            OLS Regression Results                            
Dep. Variable:          Perc_mono_Hin   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                 -0.023
Method:                 Least Squares   F-statistic:                    0.2226
Date:                Thu, 16 Mar 2023   Prob (F-statistic):              0.640
Time:                        21:39:35   Log-Likelihood:                 70.163
No. Observations:                  35   AIC:                            -136.3
Df Residuals:                      33   BIC:                            -133.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.0003      0.001     -0.4

In [43]:
aggregated_data = df[(df.CMI < 0.5) & (df.Num_Eng > df.Num_Hin)].groupby(['date'])['id'].count().reset_index(drop=False)
aggregated_data.columns = ['date','Num_mono_Eng']

aggregated_data22 = df[(df.CMI < 0.5) & (df.Num_Eng < df.Num_Hin)].groupby(['date'])['id'].count().reset_index(drop=False)
aggregated_data22.columns = ['date','Num_mono_Hin']

aggregated_data2 = df[df.CMI >= 0.5].groupby(['date'])['id'].count().reset_index(drop=False)
aggregated_data2.columns = ['date','Num_CM']

aggregated_data3 = df.groupby(['date'])['id'].count().reset_index(drop=False)
aggregated_data3.columns = ['date','Total']

aggregated_data = pd.merge(aggregated_data, aggregated_data2, how='inner')
aggregated_data = pd.merge(aggregated_data, aggregated_data3, how='inner')
aggregated_data = pd.merge(aggregated_data, aggregated_data22, how='inner')

aggregated_data['month'] = aggregated_data.date.apply(lambda x: int(x.split("/")[1]))
aggregated_data['year'] = aggregated_data.date.apply(lambda x: int(x.split("/")[2]))
aggregated_data = aggregated_data.sort_values(['year','month']).reset_index(drop=True)

aggregated_data['Perc_mono_Hin'] = aggregated_data.Num_mono_Hin/aggregated_data.Total
aggregated_data['Perc_mono_Eng'] = aggregated_data.Num_mono_Eng/aggregated_data.Total
aggregated_data['Perc_CM'] = aggregated_data.Num_CM/aggregated_data.Total
#aggregated_data['CM_logit'] = aggregated_data.Num_CM/aggregated_data.Num_mono


In [ ]:
p = figure(height=600, width=1200, x_range=aggregated_data.date.unique())

p.line(aggregated_data.date, aggregated_data.Perc_mono_Hin, legend_label='Monolingual Hindi', line_color="orange", line_dash=[4,4],line_width=2)
p.line(aggregated_data.date, aggregated_data.Perc_CM, legend_label='Code-mixed',line_width=2)
p.line(aggregated_data.date, aggregated_data.Perc_mono_Eng, legend_label='Monolingual English',line_color="black", line_width=2, line_dash=[2,2])

p.legend.visible=False 

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

p.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
p.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks

p.yaxis.major_tick_line_color = None  # turn off y-axis major ticks
p.yaxis.minor_tick_line_color = None  # turn off y-axis minor ticks

p.xaxis.major_label_text_font_size = '0pt'  # turn off x-axis tick labels
p.yaxis.major_label_text_font_size = '0pt'  # turn off y-axis tick labels

p.xaxis.major_label_text_color = None  # turn off x-axis tick labels leaving space
p.yaxis.major_label_text_color = None  # turn off y-axis tick labels leaving space 

p.output_backend = "svg"

export_svg(p, filename="/Users/victor/Downloads/figure2b.svg")

show(p)


In [44]:
df.Langugage_hueristic.value_counts()

Code-mixed    223324
Hindi         19413 
Other         13214 
English       6627  
Name: Langugage_hueristic, dtype: int64

In [45]:
temp_df = df[(df.year >= 2020)][['year','CMI','id']]
temp_df.CMI = temp_df.CMI.apply(lambda x: int(int(x*100)/10)/10)
temp_df = temp_df.groupby(['CMI'])['id'].nunique().reset_index(drop=False)
temp_df.columns = ['CMI','count']
temp_df['count'] = temp_df['count']/temp_df['count'].sum()

p = figure(x_range=(0,1), height=350)

p.vbar(x=temp_df.CMI, top=temp_df['count'], bottom=0, width=0.08)

p.xgrid.grid_line_color = None
p.y_range.start = 0

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

p.xaxis.axis_label = 'CMI'
p.yaxis.axis_label = 'Proportion'

p.xaxis.axis_label_text_font_style = "normal"
p.yaxis.axis_label_text_font_style = "normal"

show(p)

### Speaker level analysis

In [31]:
df.head()

,created_at,author_id,lang,text,id,public_metrics,withheld,script_lang,translated_text,POS,LID,CMI,Langugage_hueristic,follower_count,year,month,day,date,date2
0,2014-03-01 10:28:08+00:00,2.371767e+09,hi,अपन देशक नेता जी : कविता: चोर उचक्का नेता बनी केघोटाला पर जे घोटाला करतघैयला म जे भुर रहत ततहन पैइन कोना क भरत... http://t.co/V4RC5FUSbx,4.397087e+17,"{'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}",NaN,hindi,apan deshak netaa jee : kavita: chor uchakka netaa bunny keghotala parr jey ghotala karatghaiyala maam jey bhur rahat tatahan paine cona kaa bharat... http://t.co/v4rc5fusbx,"{""apan"": ""ADV"", ""deshak"": ""NOUN"", ""netaa"": ""VERB"", ""jee"": ""VERB"", "":"": ""X"", ""kavita"": ""NOUN"", ""chor"": ""NOUN"", ""uchakka"": ""VERB"", ""bunny"": ""NOUN"", ""keghotala"": ""VERB"", ""parr"": ""PART"", ""jey"": ""PRON"", ""ghotala"": ""VERB"", ""karatghaiyala"": ""VERB"", ""maam"": ""ADP"", ""bhur"": ""VERB"", ""rahat"": ""VERB"", ""tatahan"": ""NOUN"", ""paine"": ""VERB"", ""cona"": ""VERB"", ""kaa"": ""VERB"", ""bharat"": ""PROPN"", ""."": ""X"", ""http"": ""X"", ""/"": ""X"", ""t"": ""X"", ""co"": ""X"", ""v4rc5fusbx"": ""X""}","{""apan"": ""hin"", ""deshak"": ""hin"", ""netaa"": ""hin"", ""jee"": ""hin"", "":"": ""other"", ""kavita"": ""hin"", ""chor"": ""hin"", ""uchakka"": ""hin"", ""bunny"": ""hin"", ""keghotala"": ""hin"", ""parr"": ""hin"", ""jey"": ""hin"", ""ghotala"": ""hin"", ""karatghaiyala"": ""hin"", ""maam"": ""hin"", ""bhur"": ""hin"", ""rahat"": ""hin"", ""tatahan"": ""hin"", ""paine"": ""hin"", ""cona"": ""hin"", ""kaa"": ""hin"", ""bharat"": ""ne"", ""."": ""other"", ""http"": ""other"", ""/"": ""other"", ""t"": ""en"", ""co"": ""en"", ""v4rc5fusbx"": ""other""}",0.714286,Code-mixed,1.0,2014,1,1,1/1/2014,1/2014
1,2014-03-01 20:37:07+00:00,1.311165e+08,hi,Shri. Pranab Mukherjee: Do not change the name of Shankaracharya Hill in Kashmir http://t.co/NV6fL7SdEH via @Change4India,4.398619e+17,"{'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}",NaN,NaN,Shri. Pranab Mukherjee: Do not change the name of Shankaracharya Hill in Kashmir http://t.co/NV6fL7SdEH via @Change4India,"{""Shri"": ""PROPN"", ""."": ""X"", ""Pranab"": ""PROPN"", ""Mukherjee"": ""PROPN"", "":"": ""X"", ""Do"": ""VERB"", ""not"": ""PART_NEG"", ""change"": ""VERB"", ""the"": ""DET"", ""name"": ""NOUN"", ""of"": ""ADP"", ""Shankaracharya"": ""PROPN"", ""Hill"": ""PROPN"", ""in"": ""ADP"", ""Kashmir"": ""PROPN"", ""http"": ""X"", ""/"": ""X"", ""t"": ""X"", ""co"": ""X"", ""NV6fL7SdEH"": ""X"", ""via"": ""ADP"", ""@"": ""X"", ""Change4India"": ""X""}","{""Shri"": ""ne"", ""."": ""other"", ""Pranab"": ""ne"", ""Mukherjee"": ""ne"", "":"": ""other"", ""Do"": ""en"", ""not"": ""en"", ""change"": ""en"", ""the"": ""en"", ""name"": ""en"", ""of"": ""en"", ""Shankaracharya"": ""ne"", ""Hill"": ""ne"", ""in"": ""en"", ""Kashmir"": ""ne"", ""http"": ""other"", ""/"": ""other"", ""t"": ""en"", ""co"": ""en"", ""NV6fL7SdEH"": ""en"", ""via"": ""en"", ""@"": ""other"", ""Change4India"": ""ne""}",0.478261,English,NaN,2014,1,1,1/1/2014,1/2014
2,2014-01-01 17:37:06+00:00,8.914634e+07,hi,ae dil kis baat par khush hae tu? Din to badal jaate hae...dil nahi ...a very happy 2014..cheers!,4.184357e+17,"{'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}",NaN,NaN,ae dil kis baat par khush hae tu? Din to badal jaate hae...dil nahi ...a very happy 2014..cheers!,"{""ae"": ""DET"", ""dil"": ""NOUN"", ""kis"": ""ADJ"", ""baat"": ""NOUN"", ""par"": ""ADP"", ""khush"": ""VERB"", ""hae"": ""VERB"", ""tu"": ""PRON"", ""?"": ""X"", ""Din"": ""NOUN"", ""to"": ""PART"", ""badal"": ""VERB"", ""jaate"": ""VERB"", ""."": ""X"", ""nahi"": ""PART_NEG"", ""a"": ""DET"", ""very"": ""ADV"", ""happy"": ""ADJ"", ""2014"": ""NUM"", ""cheers"": ""VERB"", ""!"": ""X""}","{""ae"": ""hin"", ""dil"": ""hin"", ""kis"": ""hin"", ""baat"": ""hin"", ""par"": ""hin"", ""khush"": ""hin"", ""hae"": ""hin"", ""tu"": ""hin"", ""?"": 

In [46]:
df2 = df.groupby(['date2','author_id']).agg({'CMI': 'mean', 'Num_Eng': 'sum', 'Num_Hin': 'sum'}).reset_index(drop=False)
df2.columns = ['date2','author_id','CMI','Num_Eng','Num_Hin']

aggregated_data = df2[(df2.CMI < 0.5) & (df2.Num_Eng > df2.Num_Hin)].groupby(['date2'])['author_id'].nunique().reset_index(drop=False)
aggregated_data.columns = ['date2','Num_mono_Eng']

aggregated_data22 = df2[(df2.CMI < 0.5) & (df2.Num_Eng < df2.Num_Hin)].groupby(['date2'])['author_id'].nunique().reset_index(drop=False)
aggregated_data22.columns = ['date2','Num_mono_Hin']

aggregated_data2 = df2[df2.CMI >= 0.5].groupby(['date2'])['author_id'].nunique().reset_index(drop=False)
aggregated_data2.columns = ['date2','Num_CM']

aggregated_data3 = df2.groupby(['date2'])['author_id'].nunique().reset_index(drop=False)
aggregated_data3.columns = ['date2','Total']

aggregated_data = pd.merge(aggregated_data, aggregated_data2, how='inner')
aggregated_data = pd.merge(aggregated_data, aggregated_data3, how='inner')
aggregated_data = pd.merge(aggregated_data, aggregated_data22, how='inner')

aggregated_data['month'] = aggregated_data.date2.apply(lambda x: int(x.split("/")[0]))
aggregated_data['year'] = aggregated_data.date2.apply(lambda x: int(x.split("/")[1]))
aggregated_data = aggregated_data.sort_values(['year','month']).reset_index(drop=True)

aggregated_data['Perc_mono_Hin'] = aggregated_data.Num_mono_Hin/aggregated_data.Total
aggregated_data['Perc_mono_Eng'] = aggregated_data.Num_mono_Eng/aggregated_data.Total
aggregated_data['Perc_CM'] = aggregated_data.Num_CM/aggregated_data.Total
#aggregated_data['CM_logit'] = aggregated_data.Num_CM/aggregated_data.Num_mono

aggregated_data['id'] = aggregated_data.index

aggregated_data = aggregated_data[aggregated_data.year >= 2014]


In [ ]:
p = figure(height=600, width=1200, x_range=aggregated_data.date2.unique())

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = 'Proportion'
#

p.y_range = Range1d(start=-0, end=1)
p.extra_y_ranges = {"Volume": Range1d(start=-0, end=5000)}

p.add_layout(LinearAxis(y_range_name="Volume", axis_label='Number of Twitter Handles'), 'right')

p.vbar(x=aggregated_data.date2, top=aggregated_data.Total, bottom=0, width=0.5, y_range_name='Volume', color='orange')

p.line(aggregated_data.date2, aggregated_data.Perc_mono_Hin, legend_label='Monolingual Hindi', line_color="violet", line_dash=[4,4],line_width=4)
p.line(aggregated_data.date2, aggregated_data.Perc_CM, legend_label='Code-mixed',line_width=4)
p.line(aggregated_data.date2, aggregated_data.Perc_mono_Eng, legend_label='Monolingual English',line_color="black", line_width=4, line_dash=[2,2])

p.xaxis.major_label_orientation = math.pi/4

p.xaxis.axis_label_text_font_style = "normal"
p.yaxis.axis_label_text_font_style = "normal"

lower = BoxAnnotation(right=breaks_jkp[0].values[0], fill_alpha=0.1, fill_color='firebrick')
p.add_layout(lower)

lower = BoxAnnotation(left=breaks_jkp[0].values[0], right=breaks_jkp[1].values[0], fill_alpha=0.1, fill_color='navy')
p.add_layout(lower)

lower = BoxAnnotation(left=breaks_jkp[1].values[0], fill_alpha=0.1, fill_color='olive')
p.add_layout(lower)


#p.add_layout(p.legend[0], 'center')
p.legend.location = "top_left"
p.legend.click_policy="hide"

p.xaxis.axis_label_text_font_size = "20pt"
p.yaxis.axis_label_text_font_size = "20pt"

p.xaxis.major_label_text_font_size = "12pt"
p.yaxis.major_label_text_font_size = "12pt"

p.legend.title_text_font_size = '12pt'
 
# Increasing the labels of the box
# to 30pt
p.legend.label_text_font_size = "12pt"

p.output_backend = "svg"

export_svg(p, filename="/Users/victor/Downloads/figure3.svg")

show(p)


### Statistical Tests

In [47]:
aggregated_data['id'] = aggregated_data.index
for i in range(len(breaks_jkp)+1):
    if i == 0:
        print ("Between first timestep and {}".format(breaks_jkp[i].values[0]))

        aggregated_data2 = aggregated_data.iloc[0:breaks_jkp[i].values[0]].reset_index(drop=True)
        aggregated_data2['id'] = aggregated_data2.index
        
    elif i == len(breaks_jkp):
        print ("Between {} and last timestep".format(breaks_jkp[i-1].values[0]))

        aggregated_data2 = aggregated_data.iloc[breaks_jkp[i-1].values[0]:].reset_index(drop=True)
        aggregated_data2['id'] = aggregated_data2.index
        
    else:
        print ("Between {} and {} timestep".format(breaks_jkp[i-1].values[0], breaks_jkp[i].values[0]))

        aggregated_data2 = aggregated_data.iloc[breaks_jkp[i-1].values[0]:breaks_jkp[i].values[0]].reset_index(drop=True)
        aggregated_data2['id'] = aggregated_data2.index
    
    #additive_decomposition = seasonal_decompose(aggregated_data2.CMI, model='additive', period=4)
    #p = figure(height=400, width=1000, x_range=aggregated_data.date.unique())
    #p.line(aggregated_data2.date, additive_decomposition.trend, legend_label='CMI', line_dash=[4,4],line_width=2)
    #p.xaxis.major_label_orientation = math.pi/4
    #show(p)
    
    X = aggregated_data2['id'].values.reshape(-1,1)
    X = np.concatenate([X,np.asarray([1]*X.shape[0]).reshape(-1, 1)],-1)
    
    print("Mono Hindi")
    mod = sm.OLS(aggregated_data2.Perc_mono_Hin, X)
    res = mod.fit()
    print(res.summary())
    
    print("Mono English")
    mod = sm.OLS(aggregated_data2.Perc_mono_Eng, X)
    res = mod.fit()
    print(res.summary())
    
    print("CM")
    mod = sm.OLS(aggregated_data2.Perc_CM, X)
    res = mod.fit()
    print(res.summary())
    
    
    try:
        print ("Mono Hindi")
        #print (kpss(aggregated_data2.Perc_mono_Hin, regression='ct'))
        print (grangercausalitytests(aggregated_data2[['Perc_mono_Hin', 'id']], maxlag=2))
        
        print ("Mono English")
        #print (kpss(aggregated_data2.Perc_mono_Eng, regression='ct'))
        print (grangercausalitytests(aggregated_data2[['Perc_mono_Eng', 'id']], maxlag=2))
        
        print ("CM")
        #print (kpss(aggregated_data2.Perc_CM, regression='ct'))
        print (grangercausalitytests(aggregated_data2[['Perc_CM', 'id']], maxlag=2))
        
    except:
        pass
    

Between first timestep and 4
Mono Hindi
                            OLS Regression Results                            
Dep. Variable:          Perc_mono_Hin   R-squared:                       0.508
Model:                            OLS   Adj. R-squared:                  0.263
Method:                 Least Squares   F-statistic:                     2.068
Date:                Thu, 16 Mar 2023   Prob (F-statistic):              0.287
Time:                        22:11:58   Log-Likelihood:                 7.0279
No. Observations:                   4   AIC:                            -10.06
Df Residuals:                       2   BIC:                            -11.28
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          

/Users/victor/miniforge3/lib/python3.9/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 4 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
/Users/victor/miniforge3/lib/python3.9/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 4 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
/Users/victor/miniforge3/lib/python3.9/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 4 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
/Users/victor/miniforge3/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1477: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/Users/victor/

In [48]:
X = aggregated_data['id'].values.reshape(-1,1)
X = np.concatenate([X,np.asarray([1]*X.shape[0]).reshape(-1, 1)],-1)
    
print("Mono Hindi")
mod = sm.OLS(aggregated_data.Perc_mono_Hin, X)
res = mod.fit()
print(res.summary())
print (grangercausalitytests(aggregated_data[['Perc_mono_Hin', 'id']], maxlag=2))

print("Mono English")
mod = sm.OLS(aggregated_data.Perc_mono_Eng, X)
res = mod.fit()
print(res.summary())
print (grangercausalitytests(aggregated_data[['Perc_mono_Eng', 'id']], maxlag=2))

print("CM")
mod = sm.OLS(aggregated_data.Perc_CM, X)
res = mod.fit()
print(res.summary())
print (grangercausalitytests(aggregated_data[['Perc_CM', 'id']], maxlag=2))

Mono Hindi
                            OLS Regression Results                            
Dep. Variable:          Perc_mono_Hin   R-squared:                       0.110
Model:                            OLS   Adj. R-squared:                  0.083
Method:                 Least Squares   F-statistic:                     4.084
Date:                Thu, 16 Mar 2023   Prob (F-statistic):             0.0515
Time:                        22:12:17   Log-Likelihood:                 68.554
No. Observations:                  35   AIC:                            -133.1
Df Residuals:                      33   BIC:                            -130.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0012      0.001      2.0